Potential racism detection model - Bigotry_dict keyword dectection. Assuming you have already downloaded the outputs folder of deeds from the original eda.ipynb. 

In [10]:
import os
def read_from_dir(directory):
    txt_content_array = []
    
    for file_name in os.listdir(directory):
        if file_name.endswith('.txt'):
            file_path = os.path.join(directory, file_name)
            with open(file_path, 'r', encoding='utf-8') as file:
                content = file.read()
                txt_content_array.append(content)
    
    return txt_content_array

directory_path_outputs = './outputs'

outputs = read_from_dir(directory_path_outputs)
print(f'Extracted text for {len(outputs)} tiffs')

Extracted text for 600 tiffs


In [11]:
import re
import spacy
from collections import Counter

nlp = spacy.load('en_core_web_sm')

def preprocess_text(text):
    text = re.sub(r'[\n\r\t]', ' ', text)
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    doc = nlp(text)
    
    result = {
        "original_text": text,
        "sentences": [],
        "pos_groups": {},
        "named_entities": [],
        "dependencies": [],
        "token_offsets": [],
        "word_frequency": {},
        "sentence_lengths": [],
        "pos_counts": {}
    }
    
    pos_groups = {
        "NOUN": [], "VERB": [], "ADJ": [], "ADV": [], "PROPN": [],
        "DET": [], "AUX": [], "PRON": [], "ADP": [], "NUM": [],
        "PART": [], "PUNCT": [], "INTJ": [], "X": []
    }
    
    all_tokens = []
    
    for sent in doc.sents:
        result["sentences"].append(sent.text)
        result["sentence_lengths"].append(len(sent))
        
        for token in sent:
            pos = token.pos_
            all_tokens.append(token.text)
            
            if pos in pos_groups:
                pos_groups[pos].append(token.text)
                
            result["dependencies"].append({
                "token": token.text,
                "dep": token.dep_,
                "head": token.head.text
            })
            result["token_offsets"].append({
                "token": token.text,
                "start": token.idx,
                "end": token.idx + len(token.text)
            })
    
    result["pos_groups"] = pos_groups
    result["named_entities"] = [{"text": ent.text, "label": ent.label_} for ent in doc.ents]
    result["word_frequency"] = dict(Counter(all_tokens))
    result["pos_counts"] = dict(Counter([token.pos_ for token in doc]))
    
    return result

text_objects = []
for i, text in enumerate(outputs):
    text_objects.append(preprocess_text(text))
    if i % 100 == 0:
        print(f'Converted text {i} into object')

print(text_objects[0]["pos_counts"])

Converted text 0 into object
Converted text 100 into object
Converted text 200 into object
Converted text 300 into object
Converted text 400 into object
Converted text 500 into object
{'NUM': 38, 'NOUN': 219, 'PROPN': 87, 'DET': 97, 'ADP': 135, 'PUNCT': 112, 'CCONJ': 89, 'VERB': 104, 'AUX': 40, 'ADV': 37, 'ADJ': 48, 'PART': 13, 'SCONJ': 20, 'PRON': 31, 'SPACE': 2}


In [15]:
import os
import re
import pandas as pd
from bigotry_dict import bigotry_dict

def count_keywords_in_text(text, bigotry_dict):
    keyword_counts = {}
    text_lower = text.lower()  # Normalize the text to lowercase

    for keyword in bigotry_dict:
        keyword_lower = keyword.lower()  # Normalize the keyword to lowercase
        # Use regular expressions to match only whole words
        pattern = r'\b' + re.escape(keyword_lower) + r'\b'
        matches = re.findall(pattern, text_lower)
        count = len(matches)  # Count the number of whole-word matches

        if count > 0:
            # Initialize the keyword count if it's not already present
            if keyword_lower not in keyword_counts:
                keyword_counts[keyword_lower] = {'count': 0, 'texts': [], 'display_keyword': keyword}
            keyword_counts[keyword_lower]['count'] += 1  # Count the keyword only once per text
            keyword_counts[keyword_lower]['texts'].append(text)  # Add the text where the keyword appears
    
    return keyword_counts

def save_racist_deed(text, deed_id, output_dir="racist_deeds"):
    """Saves the deed text to a txt file if racist keywords are found."""
    os.makedirs(output_dir, exist_ok=True)  # Create output directory if it doesn't exist
    
    file_path = os.path.join(output_dir, f"deed_{deed_id}.txt")
    with open(file_path, 'w') as f:
        f.write(text)
    
    print(f"Deed saved to: {file_path}")

def process_deeds(text_objects):
    total_keyword_counts = {}

    for i, text_obj in enumerate(text_objects):
        keyword_counts = count_keywords_in_text(text_obj['original_text'], bigotry_dict)
        
        racist_deed = False
        for keyword_lower, data in keyword_counts.items():
            if data['count'] > 0:
                racist_deed = True  # Mark deed as racist if any keyword is found
                if keyword_lower not in total_keyword_counts:
                    total_keyword_counts[keyword_lower] = {'count': 0, 'texts': [], 'display_keyword': data['display_keyword']}
                total_keyword_counts[keyword_lower]['count'] += 1  # Ensure the keyword is only counted once per deed
                total_keyword_counts[keyword_lower]['texts'].extend(data['texts'])  # Collect texts

        # If any racist keyword is found, save the deed text
        if racist_deed:
            save_racist_deed(text_obj['original_text'], i)

    # Convert the total counts to a pandas DataFrame for easier analysis
    keyword_df = pd.DataFrame([(data['display_keyword'], data['count'], data['texts']) 
                               for data in total_keyword_counts.values()], 
                              columns=['Keyword', 'Count', 'Texts'])

    # Sort keywords by count for analysis
    keyword_df_sorted = keyword_df.sort_values(by="Count", ascending=False)

    # Display the dataframe for analysis (texts associated with each keyword)
    print(keyword_df_sorted)

process_deeds(text_objects)


Deed saved to: racist_deeds\deed_53.txt
Deed saved to: racist_deeds\deed_67.txt
Deed saved to: racist_deeds\deed_95.txt
Deed saved to: racist_deeds\deed_99.txt
Deed saved to: racist_deeds\deed_152.txt
Deed saved to: racist_deeds\deed_182.txt
Deed saved to: racist_deeds\deed_186.txt
Deed saved to: racist_deeds\deed_266.txt
Deed saved to: racist_deeds\deed_281.txt
Deed saved to: racist_deeds\deed_301.txt
Deed saved to: racist_deeds\deed_308.txt
Deed saved to: racist_deeds\deed_309.txt
Deed saved to: racist_deeds\deed_320.txt
Deed saved to: racist_deeds\deed_356.txt
Deed saved to: racist_deeds\deed_371.txt
Deed saved to: racist_deeds\deed_389.txt
Deed saved to: racist_deeds\deed_418.txt
Deed saved to: racist_deeds\deed_432.txt
Deed saved to: racist_deeds\deed_467.txt
Deed saved to: racist_deeds\deed_523.txt
Deed saved to: racist_deeds\deed_531.txt
Deed saved to: racist_deeds\deed_572.txt
    Keyword  Count                                              Texts
1     white     19  [i 68 See Du